In [6]:
import subprocess
import time
import sys

def get_signal_strength():
    """Measure Wi-Fi signal strength using netsh and return percentage (or None if unavailable)."""
    try:
        result = subprocess.check_output("netsh wlan show interfaces", shell=True, text=True)
        print(f"\nDebug netsh Output:\n{result}")  # Debug: Print raw netsh output
        for line in result.splitlines():
            if "Signal" in line:
                percentage = int(line.split(":")[1].strip().replace("%", ""))
                return percentage
        return None
    except Exception as e:
        print(f"Error getting signal strength: {e}")
        return None

def percentage_to_dbm(percentage):
    """Convert signal strength percentage to approximate dBm."""
    if percentage is None or percentage == 0:
        return -100  # Assume -100 dBm if no signal
    # Rough approximation: 100% ≈ -30 dBm, 0% ≈ -100 dBm (linear mapping)
    dbm = -100 + (percentage * 0.7)  # Maps 0% to -100 dBm, 100% to -30 dBm
    return round(dbm)

def ping_host(host, attempts=40):
    results = []
    print(f"Pinging {host} up to {attempts} times...")
    
    for i in range(attempts):
        # Run ping command and capture output
        ping_process = subprocess.run(['ping', '-n', '1', '-w', '2000', host], 
                                     stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        
        # Debug: Print raw output to see what's being captured
        print(f"\nDebug Output for Attempt {i+1}:\n{ping_process.stdout}")
        if ping_process.stderr:
            print(f"Error Output: {ping_process.stderr}")

        # Check if ping was successful
        if "Reply from" in ping_process.stdout:
            # Extract response time
            for line in ping_process.stdout.splitlines():
                if "time=" in line or "time<" in line:  # Handle both "time=1ms" and "time<1ms"
                    try:
                        if "time<" in line:
                            response_time = 0.5  # Approximate for "time<1ms"
                        else:
                            response_time = float(line.split("time=")[1].split("ms")[0])
                        status = "Success"
                        break
                    except (IndexError, ValueError):
                        response_time = 0
                        status = "Failed"
                        break
            else:
                response_time = 0
                status = "Failed"
        else:
            response_time = 0
            status = "Failed"
        
        # Measure signal strength for this attempt
        signal_percentage = get_signal_strength()
        signal_dbm = percentage_to_dbm(signal_percentage)
        
        results.append((timestamp, response_time, status, signal_dbm))
        print(f"Attempt {i+1}: {status}, Response Time: {response_time:.2f} ms, Signal Strength: {signal_dbm} dBm" 
              if status == "Success" else f"Attempt {i+1}: {status}, Signal Strength: {signal_dbm} dBm")
        
        time.sleep(1)  # Delay between pings

    return results

def generate_text_report(results, host):
    # Generate report content as plain text for command-line output
    report = f"Ping Report for Host: {host}\n"
    report += f"Date: {time.strftime('%Y-%m-%d %H:%M:%S')}\n\n"
    report += "Ping Test Results:\n"
    report += "Attempt | Timestamp          | Response Time (ms) | Status  | Signal Strength (dBm)\n"
    report += "-" * 80 + "\n"
    
    successful_pings = sum(1 for _, _, status, _ in results if status == "Success")
    total_response_time = sum(r[1] for r in results if r[1] > 0)
    avg_response_time = total_response_time / successful_pings if successful_pings > 0 else 0
    failed_pings = len(results) - successful_pings
    avg_signal_dbm = sum(r[3] for r in results) / len(results) if results else -100

    for i, (timestamp, response_time, status, signal_dbm) in enumerate(results, 1):
        report += f"{i:<7} | {timestamp} | {response_time:<18.2f} | {status:<7} | {signal_dbm:<19}\n"
    
    report += "\nSummary:\n"
    report += f"Total Attempts: {len(results)}\n"
    report += f"Successful Pings: {successful_pings}\n"
    report += f"Failed Pings: {failed_pings}\n"
    report += f"Average Response Time: {avg_response_time:.2f} ms\n"
    report += f"Average Signal Strength: {avg_signal_dbm:.2f} dBm\n"

    # Save to file
    with open("ping_report.txt", "w") as f:
        f.write(report)
    
    # Print report to console
    print(report)
    print("Text report generated: ping_report.txt")

# Host selection for command-line usage
default_host = "10.200.200.210"
host = default_host

# Check for command-line arguments
if len(sys.argv) > 1:
    host = sys.argv[1]

# Run the ping and generate report
results = ping_host(host)
generate_text_report(results, host)

Pinging -f up to 40 times...

Debug Output for Attempt 1:
IP address must be specified.

Error getting signal strength: Command 'netsh wlan show interfaces' returned non-zero exit status 1.
Attempt 1: Failed, Signal Strength: -100 dBm

Debug Output for Attempt 2:
IP address must be specified.

Error getting signal strength: Command 'netsh wlan show interfaces' returned non-zero exit status 1.
Attempt 2: Failed, Signal Strength: -100 dBm

Debug Output for Attempt 3:
IP address must be specified.

Error getting signal strength: Command 'netsh wlan show interfaces' returned non-zero exit status 1.
Attempt 3: Failed, Signal Strength: -100 dBm

Debug Output for Attempt 4:
IP address must be specified.

Error getting signal strength: Command 'netsh wlan show interfaces' returned non-zero exit status 1.
Attempt 4: Failed, Signal Strength: -100 dBm

Debug Output for Attempt 5:
IP address must be specified.

Error getting signal strength: Command 'netsh wlan show interfaces' returned non-zero e